# Generator File: GPT2-Small Finetuned incl. unfreezing


## 1. Importing libraries and functions

In [ ]:
try:
    print(gpt2_libraries_progress)
except NameError:
    %run ./generation/generator_gpt2_libraries.ipynb

## 2. Importing model

In [ ]:
print("Loading model...")

In [ ]:
# Split a GPT2 model in 4 groups for differential learning rates (Code from Finetuning English GPT2 to any language)

def splitter(model):
    "Split a GPT2 `model` in 3 groups for differential learning rates."
    
    # First layers group : decoder blocks from 0 to 3
    modules = []
    for i in range(4): modules.append(model.transformer.h[i])
    groups = [nn.Sequential(*modules)]

    # Second layers group : decoder blocks from 4 to 7
    modules = []
    for i in range(4,8,1): modules.append(model.transformer.h[i])
    groups = L(groups + [nn.Sequential(*modules)])

    # Third layers group : decoder blocks from 8 to 11
    modules = []
    for i in range(8,12,1): modules.append(model.transformer.h[i])
    groups = L(groups + [nn.Sequential(*modules)])
    
    # Fourth layers group : embeddings matrices wte and wpe + LayerNorm at the model output
    groups = L(groups + [nn.Sequential(model.transformer.wte,model.transformer.wpe,model.transformer.ln_f)])
    
    return groups.map(params)

In [ ]:
# Load model
gpt2S_tuned_on_tandc_unfrozen = load_learner(model_path + "gpt2S_tuned_on_tandc_unfrozen.pkl")

In [ ]:
# Define tokenizer
gpt2s_tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', add_prefix_space=True)

##3. Loading functions

In [ ]:
def gen_story_gpt2s_tunedonTC_unfrozen(seed, max_len):
  return gen_story(my_model = gpt2S_tuned_on_tandc_unfrozen, my_tokenizer = gpt2s_tokenizer, seed=seed, max_len=max_len) 

In [ ]:
clear_output()
print("Function and Model now available for use:")
print("    gen_story_gpt2s_tunedonTC_unfrozen(seed, max_len)")
print("    *Based on GPT2-Small  | Tuned on input_stories_toddlerpluschildren.txt | Unfrozen")